In [ ]:
import os
import torch as tr
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from utilsWM import *
%load_ext autoreload
%autoreload 2

In [ ]:
from utilsWM import run_model_for_epochs, plot_train_accuracy, eval_by_ttype, plot_accuracy_by_trial_type 

# main wrapper 
def train_net_and_plot_accuracy(neps, seed, model_path, figure_path):
    """ 
    main wrapper for running experiment and plotting
    """
    
    np.random.seed(seed)
    tr.random.manual_seed(seed)
    
    ## setup task
    ssizeL = [2, 3] 
    sample_fn = lambda match,slure,clure: lambda S,C,N: single_nback_comparison(S,C,N,
              pr_match=match,pr_stim_lure=slure,pr_context_lure=clure
              )

    taskintL = [0,1]
    taskL_tr = [
         [taskintL[0],sample_fn(PMATCH,PSLURE,PCLURE),ssizeL[0]],
         [taskintL[1],sample_fn(PMATCH,PSLURE,PCLURE),ssizeL[1]],
        ]

    # init net
    indim = 2 * (CDIM + SDIM)
    hiddim = SDIM * 4
    net = FFWM(indim,hiddim)

    # train net
    score_tr, ttype = run_model_for_epochs(net,taskL_tr,
    training=True,neps_per_task=neps,
    )
    
    # save trained model
    np.save(model_path + "/train-score", score_tr)
    tr.save(net.state_dict(), model_path + "/trained-net.pt")

    # generates evaluation plots
    plot_train_accuracy(score_tr, ttype, ssizeL, figure_path)
    scores_by_ttype = eval_by_ttype(net, sample_fn, taskintL, ssizeL, neps=1000)
    plot_accuracy_by_trial_type(scores_by_ttype, taskintL, ssizeL, figure_path)

    return net



In [ ]:
# run 
n_training_eps = 1000 ## 1000000
seed = 0 ## 0

figure_path = "OUTPUT_PATH"

ffwm = train_net_and_plot_accuracy(neps=n_training_eps, seed=seed,
            model_path=model_path, figure_path=figure_path)
